In [1]:
import pyfor
import matplotlib.pyplot as plt

## Normalization - The Basics

One of the most integral parts of LiDAR analysis is normalizing the point cloud. This is done first by filtering ground points using one of many algorithms. **pyfor** currently implements the progressive morphological ground filter (Zhang et al. 2003) to achieve this.

There are two ways to handle normalization. The first is to normalize the Cloud object. This will normalize the point cloud **in place**. We will soon see that this is just a wrapper for the lower level Grid functions.

In [2]:
# Load the point cloud
# CHANGEME
pc = pyfor.cloud.Cloud("my_las.las")

# Plot the raw cloud
#pc.iplot3d(point_size=2)

<img src="images/raw_iplot.png" width="550">

In [13]:
# Call the normalization functions with some useful settings for the algorithm.
pc.normalize(cell_size=0.4, num_windows=7, dh_0= 1, dh_max=2.5)

<img src="images/norm_iplot.png" width="550">

As we can see, the point cloud is now normalized and ready for metric extraction.

## Digging Deeper

What would normalization be without a ground filter? Let's investigate the behavior of the ground filter below by digging deeper into pyfor.

First, we need to set a coordinate reference system to the point cloud.

The implementation of the Zhang et al. (2003) algorithm in pyfor is based off of a rasterized point cloud, specifically the minimum value of the `z` dimension in each cell, with missing values (i.e. empty cells) interpolated using a nearest neighbor algorithm. This is what the interpolated raster looks like: 

In [7]:
#  Reload an unnormalized point cloud
#pc = pyfor.cloud.Cloud("my_las.las")
pc_grid = pc.grid(0.5)

# Plot the interpolated array.
interp_min = pc_grid.interpolate("min", "z")
#interp_min.plot()

Clearly this is not the final product we desire. Just because a given minimum z value exists in a given cell does not mean that point represents the ground. This is where the progressive morphological filter kicks in: we need to filter these "unlikely ground" points from the raster. Here we can directly call the `ground_filter` implementation for our `pc_grid` Grid object. The argument values should look familiar.

In [8]:
bem = pc_grid.ground_filter(7, 2.5, 1)
#bem.plot()

If we are satisfied with the performance of our ground filter, we can save the bare earth model to disk.